In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import scipy.stats as stats
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler,FunctionTransformer,MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

%matplotlib inline 

# Load the data

In [ ]:
df=pd.read_csv("../input/insurance/insurance.csv")
df.shape

In [ ]:
#check the data 
df.head()

In [ ]:
#check the data information
df.info()


In [ ]:
#checking the null values
df.isnull().sum()

there is no null value on the columns

In [ ]:
#checking the data discription(numerical col only)
df.describe()

In [ ]:
#checking corelation among the pair of columns 
df.corr()

In [ ]:
sns.heatmap(df.corr(),cmap='plasma',annot=True)

here we observed charges is increased when age,bmi are increased  proptionally

# Applying EDA for data analysis 

In [ ]:
df.head()

In [ ]:
#analysis sex col 
plt.figure(figsize=(18,8))
plt.subplot(121)
sns.countplot(df["sex"])

plt.subplot(122)
df["sex"].value_counts().plot(kind="pie",autopct="%.2f",shadow=True, startangle=90)
plt.show()

males are 50.52% and females are 49.48%

In [ ]:
#analysis smoker col 
plt.figure(figsize=(18,8))
plt.subplot(121)
sns.countplot(df["smoker"])

plt.subplot(122)
df["smoker"].value_counts().plot(kind="pie",autopct="%.2f",shadow=True, startangle=90)
plt.show()

smoker are 79.52% and non-smoker are 20.48%

In [ ]:
#analysis region col 
plt.figure(figsize=(18,8))
plt.subplot(121)
sns.countplot(df["region"])

plt.subplot(122)
df["region"].value_counts().plot(kind="pie",autopct="%.2f",shadow=True, startangle=90)
plt.show()

In [ ]:
#analysis sex vs smoker columns combinely 
sns.countplot(df["sex"],hue=df["smoker"])

we got some female smoker and male smoker but also we realized that most of the female are non smoker compare to male 

In [ ]:
#anlysis numerical columns
sns.pairplot(df)

In [ ]:
#analysis age and bmi columns
plt.figure(figsize=(18,8))
plt.subplot(121)
plt.hist(df["age"],color="green")
plt.title("Age")
plt.xlabel("age")
plt.ylabel("count")

plt.subplot(122)
plt.hist(df["bmi"],color="red")
plt.title("BMI")
plt.xlabel("bmi")
plt.ylabel("count")

plt.show()


In [ ]:
#analysis childern and charges columns  
plt.figure(figsize=(28,10))
plt.subplot(121)
plt.hist(df["children"],color="blue")
plt.title("Children")
plt.xlabel("children")
plt.ylabel("count")

plt.subplot(122)
plt.hist(df["charges"],color="black")
plt.title("Charges")
plt.xlabel("charges")
plt.ylabel("count")

plt.show()


In [ ]:
#analysis age vs bmi columns with smoker also
plt.figure(figsize=(24,14))
sns.jointplot(df["age"],df["bmi"],hue=df["smoker"])



In [ ]:
#analysis age vs charges columns with sex also
sns.jointplot(df["age"],df["charges"],hue=df["sex"])



In [ ]:
#numerical - categorical col analysis
plt.figure(figsize=(18,8))
plt.subplot(121)
sns.barplot(df["sex"],df["charges"],hue=df["smoker"])

plt.subplot(122)
sns.barplot(df["region"],df["bmi"],hue=df["sex"])
plt.show()


In [ ]:
#categorical-categorical
plt.figure(figsize=(18,8))
plt.subplot(121)
sns.heatmap(pd.crosstab(df["sex"],df["region"]))

plt.subplot(122)
sns.heatmap(pd.crosstab(df["sex"],df["smoker"]),cmap='plasma')
plt.show()


In [ ]:
#creating pivot table
pivot=df.pivot_table(values="charges",index="children",columns="region")
sns.heatmap(pivot)

# Feauture Engineeering

# 1. outlier detection

In [ ]:
#checking for outlier in age col
sns.boxplot(df["age"])

all is well ,no outlier is detected

In [ ]:
#outliers in bmi col
sns.boxplot(df["bmi"])

oho,some outlier is found in bmi col.

In [ ]:
#first check the distribution
sns.distplot(df["bmi"],kde=True,color="green")

In [ ]:
#let solve the bmi outlier 
#we can using Z-score for outlier detection because it's normally distributed
print("mean vlaue:",df["bmi"].mean())
print("std vlaue:",df["bmi"].std())
print("min vlaue:",df["bmi"].min())
print("max vlaue:",df["bmi"].max())

In [ ]:
#let find the boundaries for outlier
print("highest :",df["bmi"].mean() + 3*df["bmi"].std())
print("Lowest :",df["bmi"].mean() - 3*df["bmi"].std())

In [ ]:
#finally, let find the outlier
outliers=df[(df["bmi"]>48.957) |(df["bmi"]<12.3688)]
outliers

There ara 4 potential outlier in bmi col ,we can hunle this by trimming(removing)

In [ ]:
#remove the outlier
df=df[(df["bmi"]<48.957) & (df["bmi"]>12.3688)]
df.shape


the outliers are removed 

In [ ]:
#function that will change  bmi to a category
def weightCondition(bmi):
  if bmi<18.5:
    return "Underweight"
  elif (bmi>= 18.5)&(bmi< 24.986):
    return "Normal"
  elif (bmi >= 25) & (bmi < 29.926):
    return "Overweight"
  else:
    return "Obese"
df["weight_Condition"]=[weightCondition(val) for val in df["bmi"] ]
df.head(5)

In [ ]:
#get the features and terget col
Y=df.charges
X=df.drop(["charges"],axis=1)
#train test split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

# 2.Encoding with OHE +Ordinal Encoder and scaling 

In [ ]:
x_train.head()

In [ ]:
x_train["weight_Condition"].unique()

In [ ]:
#pipe1 contain 2 encoder ,one hot encoder and ordinal encoder
#ohe encode sex,smoker,region
#ordinal encode the weight_condition col because we arrange the order on this col
#pipe2 just scale all the columns 
pipe1=ColumnTransformer(transformers=[("OHE",OneHotEncoder(sparse=False,drop="first"),["sex","smoker","region"]),
                                     ("ordinal",OrdinalEncoder(categories=[['Underweight','Normal','Overweight','Obese']]),["weight_Condition"])]
                                           ,remainder="passthrough")
pipe2=ColumnTransformer(transformers=[("scaling",StandardScaler(),[0,1,2,3,4,5,6,7,8])]
                                            ,remainder="passthrough")

pipe=Pipeline([("pipe1",pipe1),("pipe2",pipe2)])
x_train=pd.DataFrame(pipe.fit_transform(x_train))
x_test=pd.DataFrame(pipe.transform(x_test))
x_train.head()

all the columns are scalled 

# evaluate the best models 

In [ ]:
#linear regression
lr=LinearRegression()
lr.fit(x_train,y_train)
ypred_lr=lr.predict(x_test)
print("R2 score",r2_score(y_test,ypred_lr))

In [ ]:
#Random forest regressor
rf=RandomForestRegressor(criterion='mae',n_estimators=150,max_depth=8,random_state=42)
rf.fit(x_train,y_train)
ypred_rf=rf.predict(x_test)
print("R2 score",r2_score(y_test,ypred_rf))

In [ ]:
#knn regressor
knn=KNeighborsRegressor(metric ='minkowski',n_neighbors= 9, weights= 'distance')
knn.fit(x_train,y_train)
ypred_knn=knn.predict(x_test)
print("R2 score",r2_score(y_test,ypred_knn))

In [ ]:
#Adaboost regressor
ada=AdaBoostRegressor(learning_rate= 0.001,loss='exponential',n_estimators= 50,random_state=42)
ada.fit(x_train,y_train)
ypred_ada=ada.predict(x_test)
print("R2 score",r2_score(y_test,ypred_ada))

In [ ]:
#Gradiant boost regressor
gbr=GradientBoostingRegressor(criterion= 'mse',learning_rate= 0.05, loss="ls",n_estimators= 100,random_state=42)
gbr.fit(x_train,y_train)
ypred_gbr=gbr.predict(x_test)
print("R2 score",r2_score(y_test,ypred_gbr))

# our 1st best model is Gradiant boost regressor
# 2nd best model is Adaboost regressor
# worst model is Linear regressor